In [1]:
!pip install -r /kaggle/input/depthanythingv2/transformers/default/1/Depth-Anything-V2/requirements.txt

In [2]:
import os, gc, cv2, sys, torch
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchvision.transforms as T
import scipy.io
from PIL import Image

# ========== 模型加载 ==========
sys.path.append("/kaggle/input/depthanythingv2/transformers/default/1/Depth-Anything-V2")
from depth_anything_v2.dpt import DepthAnythingV2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DepthAnythingV2(encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024])
model.load_state_dict(torch.load(
    '/kaggle/input/depthanythingv2/transformers/default/1/Depth-Anything-V2/checkpoints/depth_anything_v2_vitl.pth',
    map_location=device
))
model.to(device)
model.eval()

# ========== 路径配置 ==========
mat_path = '/kaggle/input/nyuv2-dataset/nyu_depth_v2_labeled.mat'
save_dir = "/kaggle/working/"
os.makedirs(save_dir, exist_ok=True)
vis_dir = os.path.join(save_dir, "visualizations")
os.makedirs(vis_dir, exist_ok=True)
pres_dir = os.path.join(save_dir, "pres")
os.makedirs(pres_dir, exist_ok=True)

# ====== 加载NYUv2数据 ======
with h5py.File(mat_path, 'r') as file:
    images = file['images'][:]    # [3, H, W, N] (480, 640, 3, 1449)
    depths = file['depths'][:]    # [H, W, N] (480, 640, 1449)
#print("raw images shape:", images.shape)
#print("raw depths shape:", depths.shape)
#raw images shape: (1449, 3, 640, 480)
#raw depths shape: (1449, 640, 480)
images = np.transpose(images, (3, 2, 1, 0))  # [H, W, 3, N]
depths = np.transpose(depths, (2, 1, 0))     # [H, W, N]

print("after transpose images shape:", images.shape)  # (480, 640, 3, 1449)
print("after transpose depths shape:", depths.shape)  # (480, 640, 1449)

# ====== 官方Test split索引 ======
#idx_list = np.arange(images.shape[3])  # 全部
idx_list = np.loadtxt('/kaggle/input/nyu2-test/nyu2_test.txt').astype(int) - 1

# ====== 评估函数 ======
def compute_abs_rel(pred, gt, mask=None):
    if mask is None:
        valid = (gt > 0)
    else:
        valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    if len(gt) == 0:
        return np.nan
    return np.mean(np.abs(pred - gt) / gt)

def compute_delta1(pred, gt, mask):
    valid = mask & (gt > 1e-6) & (pred > 1e-6)
    pred, gt = pred[valid], gt[valid]
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    pred, gt = pred[mask], gt[mask]
    return np.sum(gt * pred) / np.sum(pred ** 2)

# ====== 可视化函数 ======
def save_vis(rgb, gt, pred, mask, save_path):
    plt.figure(figsize=(15, 5))
    valid = mask & (gt > 0) & np.isfinite(gt) & np.isfinite(pred)
    all_depths = np.concatenate([gt[valid], pred[valid]])
    vmin = np.percentile(all_depths, 2)
    vmax = np.percentile(all_depths, 98)

    plt.subplot(1, 3, 1)
    plt.imshow(rgb)
    plt.title("RGB")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(gt, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pred, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# ====== 图像预处理 ======
transform = T.Compose([
    T.Resize((518, 518)),  # Depth Anything 默认输入分辨率
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ====== 主评测流程 ======
results = []
for idx in tqdm(idx_list, desc="Evaluating NYUv2"):
    try:
        rgb = images[:, :, :, idx]           # [H, W, 3], uint8
        gt_depth = depths[:, :, idx]         # [H, W], float32，单位米
        #print(f"idx={idx}, rgb shape={rgb.shape}, dtype={rgb.dtype}")

        mask = (gt_depth > 0) & np.isfinite(gt_depth)

        # 预处理
        rgb_pil = Image.fromarray(rgb.astype(np.uint8))
        input_tensor = transform(rgb_pil).unsqueeze(0).to(device)

        # 推理
        with torch.no_grad():
            pred = model(input_tensor)
        pred_depth = pred.squeeze().cpu().numpy()

        # Resize 到GT尺寸
        if pred_depth.shape != gt_depth.shape:
            pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

        # 缩放匹配
        scale = scale_match(pred_depth, gt_depth, mask)
        pred_aligned = pred_depth * scale

        # 评测
        absrel = compute_abs_rel(pred_aligned, gt_depth, mask)
        delta1 = compute_delta1(pred_aligned, gt_depth, mask)
        results.append([idx, absrel, delta1])
        print(f"[{idx}] AbsRel: {absrel:.4f}, δ1: {delta1 * 100:.2f}%")

        # 可视化和保存
        vis_path = os.path.join(vis_dir, f"{idx:04d}_vis.png")
        save_vis(rgb, gt_depth, pred_aligned, mask, vis_path)
        np.save(os.path.join(pres_dir, f"{idx:04d}_pred.npy"), pred_aligned)

        del rgb, gt_depth, pred, pred_aligned, input_tensor
        torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"跳过 {idx}，错误：{e}")
        continue

# 保存结果
df = pd.DataFrame(results, columns=["idx", "AbsRel", "Delta1"])
df["original_idx"] = idx_list + 1 
mean_absrel = df["AbsRel"].mean()
mean_delta1 = df["Delta1"].mean()
mean_row = pd.DataFrame([["mean", mean_absrel, mean_delta1]], columns=["idx", "AbsRel", "Delta1"])
df = pd.concat([df, mean_row], ignore_index=True)
df.to_csv(os.path.join(save_dir, "eval_results.csv"), index=False)
print(f"\n所有评估结果保存至：{os.path.join(save_dir, 'eval_results.csv')}")

# 可选：打包结果文件夹
import shutil
vis_zip_path = os.path.join(save_dir, "visualizations.zip")
shutil.make_archive(base_name=vis_zip_path.replace(".zip", ""), format='zip', root_dir=vis_dir)
pres_zip_path = os.path.join(save_dir, "pres.zip")
shutil.make_archive(base_name=pres_zip_path.replace(".zip", ""), format='zip', root_dir=pres_dir)
print(f"visualizations 已打包至: {vis_zip_path}")
print(f"pres 已打包至: {pres_zip_path}")


after transpose images shape: (480, 640, 3, 1449)
after transpose depths shape: (480, 640, 1449)


Evaluating NYUv2:   0%|          | 0/666 [00:00<?, ?it/s]

idx=1, rgb shape=(480, 640, 3), dtype=uint8
[1] AbsRel: 0.5989, δ1: 12.23%


Evaluating NYUv2:   0%|          | 1/666 [00:01<15:41,  1.42s/it]

idx=6, rgb shape=(480, 640, 3), dtype=uint8
[6] AbsRel: 0.7741, δ1: 4.42%


...

idx=1444, rgb shape=(480, 640, 3), dtype=uint8
[1444] AbsRel: 0.7458, δ1: 11.88%


Evaluating NYUv2: 100%|█████████▉| 665/666 [26:48<00:03,  3.79s/it]

idx=1447, rgb shape=(480, 640, 3), dtype=uint8
[1447] AbsRel: 0.9299, δ1: 3.18%


Evaluating NYUv2: 100%|██████████| 666/666 [26:52<00:00,  2.42s/it]



所有评估结果保存至：/kaggle/working/eval_results.csv
visualizations 已打包至: /kaggle/working/visualizations.zip
pres 已打包至: /kaggle/working/pres.zip
